In [1]:
pip install Wikipedia-API

  Preparing metadata (setup.py) ... done
  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.7.3-py3-none-any.whl size=14325 sha256=947b524d79f6a051db18c0c2c39bea28079deca75c7b0f76d34d9cf58b7cc098
  Stored in directory: /root/.cache/pip/wheels/39/c8/cd/ba9a385ad2f6e457226fe2a7dceaeb0c17a19346d2e78ff4be
Successfully built Wikipedia-API


In [2]:
import pandas as pd
import os
import wikipediaapi

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
chemin_donnees = "/content/drive/MyDrive/PIP2025/Donnees"
path1 = os.path.join(chemin_donnees, "Meilleures audiences", "Updated_Combined_Data_Audience.csv")
path2 = os.path.join(chemin_donnees, "Données externes", "TMDb_Dataset.csv")
df = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df

,chaine,nom film,date de diffusion,2023,nationalité2,rang de diffusion,téléspectateurs (millions),part d'audience (%)
0,France 2,ANTOINETTE DANS LES CEVENNES,2023-05-21 00:00:00,2023,FRANCE,1,5758382,"27,5"
1,France 2,BOITE NOIRE,2023-11-19 00:00:00,2023,FRANCE,1,5523346,"26,6"
2,France 2,LES VETOS,2023-02-19 00:00:00,2023,FRANCE,1,4657500,22
3,France 2,LA DARONNE,2023-03-19 00:00:00,2023,FRANCE,1,4197500,20
4,France 2,LES AVENTURES DE RABBI JACOB,2023-10-22 00:00:00,2023,FRANCE,20,4054371,"17,9"
...,...,...,...,...,...,...,...,...
2206,M6,Corrina Corrina,2013-01-11 00:00:00,1999,Etats-Unis,1,"12,1","27,3"
2207,M6,Turner et Hooch,2013-02-28 00:00:00,1999,Etats-Unis,3,"11,6","25,4"
2208,M6,La Mutante,2013-03-29 00:00:00,1999,Etats-Unis,1,"9,8","22,3"
2209,M6,Opération dragon,2013-02-15 00:00:00,1999,Etats-Unis,5,"9,8","20,9"


Stratégie:
On parcourt tous les films du df des meilleures audiences
Pour chaque film:
- Si on le retrouve dans les données externes, on récupère cette ligne
- Sinon on le traduit en anglais et on récupère cette ligne


In [6]:
import requests
import wikipediaapi

# Fonction pour effectuer une recherche sur Wikipédia
def search_wikipedia(query, lang="fr"):
    URL = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "opensearch",
        "search": query,
        "limit": 1,         # On prend le 1er résultat
        "namespace": 0,
        "format": "json"
    }
    headers = {
        "User-Agent": "MonScript/1.0 (https://votre-site.com/contact)"
    }
    response = requests.get(URL, params=params, headers=headers)
    data = response.json()
    if len(data) >= 2 and data[1]:
        return data[1][0]
    return None


wiki_fr = wikipediaapi.Wikipedia(
    language='fr',
    user_agent='MonScript/1.0 (https://votre-site.com/contact)'
)

colonnes_sup = ["chaine", "date de diffusion", "nationalité2",
                "téléspectateurs (millions)", "part d'audience (%)"]
colonnes_finales = df2.columns.tolist() + colonnes_sup
df_final = pd.DataFrame(columns=colonnes_finales)

found = 0
not_found = 0

for index, row in df.iterrows():
    print(f'\r{index}/{len(df)}\t\t',end='')
    titre_fr = row['nom film']

    matching_rows = df2[df2['Title'] == titre_fr]
    if not matching_rows.empty:
        row_to_add = matching_rows.iloc[[0]].copy()
        for col in colonnes_sup:
            row_to_add[col] = row[col] if col in row.index else None
        df_final = pd.concat([df_final, row_to_add], ignore_index=True)
    else:
        requete = f"{titre_fr}"
        titre_recherche = search_wikipedia(requete, lang="fr")

        if titre_recherche:
            page_fr = wiki_fr.page(titre_recherche)
            if page_fr.exists():
              if 'en' in page_fr.langlinks:
                  page_en = page_fr.langlinks['en']
                  titre_en = page_en.title
                  matching_rows = df2[df2['Title'] == titre_en]
                  if not matching_rows.empty:
                      row_to_add = matching_rows.iloc[[0]].copy()
                      for col in colonnes_sup:
                          row_to_add[col] = row[col] if col in row.index else None
                      df_final = pd.concat([df_final, row_to_add], ignore_index=True)
                      found += 1
            else:
                not_found += 1
        else:
            not_found += 1

print("Nombre de pages trouvées :", found)
print("Nombre de pages non trouvées :", not_found)



0/2211		

<ipython-input-6-441c357175c3>:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, row_to_add], ignore_index=True)


4/2211		

<ipython-input-6-441c357175c3>:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, row_to_add], ignore_index=True)


2210/2211		Nombre de pages trouvées : 882
Nombre de pages non trouvées : 94


In [7]:
df_final

,TMDb_id,IMDB_id,Title,Categories,Date,Country,Runtime,Cast,Crew,First_Man_Age,...,Films_Collection,Original,Keywords,Poster,Bechdel,chaine,date de diffusion,nationalité2,téléspectateurs (millions),part d'audience (%)
0,669664,11013434,"My Donkey, My Lover & I","['Comedy', 'Adventure', 'Romance']",2020-09-16,FR,97,"[{'adult': False, 'gender': 1, 'id': 983929, '...","[{'adult': False, 'gender': 1, 'id': 7296, 'kn...",NaN,...,NaN,Original,[],https://image.tmdb.org/t/p/w185/hHU3tev4ujbaEl...,NaN,France 2,2023-05-21 00:00:00,FRANCE,5758382,"27,5"
1,760,69747,The Mad Adventures of Rabbi Jacob,['Comedy'],1973-10-17,Inconnu,100,"[{'adult': False, 'gender': 2, 'id': 11187, 'k...","[{'adult': False, 'gender': 2, 'id': 1657, 'kn...",59.0,...,NaN,Original,"[{'id': 90, 'name': 'paris, france'}, {'id': 2...",https://image.tmdb.org/t/p/w185/oYuCFxS2BGCHNG...,2.0,France 2,2023-10-22 00:00:00,FRANCE,4054371,"17,9"
2,370172,2382320,No Time to Die,"['Adventure', 'Action', 'Thriller']",2021-09-29,US,163,"[{'adult': False, 'gender': 2, 'id': 8784, 'kn...","[{'adult': False, 'gender': 2, 'id': 947, 'kno...",53.0,...,"[{'adult': False, 'backdrop_path': '/v28YYcN5p...",Sequel/Prequel,"[{'id': 351, 'name': 'poison'}, {'id': 470, 'n...",https://image.tmdb.org/t/p/w185/iUgygt3fscRoKW...,NaN,France 2,2023-12-10 00:00:00,GRANDE BRETAGNE / ETATS UNIS,3936853,"21,2"
3,343611,3393786,Jack Reacher: Never Go Back,"['Action', 'Thriller']",2016-10-19,US,118,"[{'adult': False, 'gender': 2, 'id': 500, 'kno...","[{'adult': False, 'gender': 2, 'id': 500, 'kno...",54.0,...,"[{'adult': False, 'backdrop_path': '/5QEtCBM6a...",Sequel/Prequel,"[{'id': 769, 'name': 'falsely accused'}, {'id'...",https://image.tmdb.org/t/p/w185/bvCEEs5l3ylNIg...,3.0,France 2,2023-01-22 00:00:00,ETATS UNIS,3507500,"16,8"
4,39386,380349,The Corsican File,['Comedy'],2004-10-06,FR,92,"[{'adult': False, 'gender': 2, 'id': 28781, 'k...","[{'adult': False, 'gender': 2, 'id': 2949, 'kn...",52.0,...,NaN,Original,[],https://image.tmdb.org/t/p/w185/aeYtaowpQTgp5O...,NaN,France 3,2023-02-20 00:00:00,FRANCE,2357500,"10,6"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,114,100405,Pretty Woman,"['Romance', 'Comedy']",1990-03-23,US,119,"[{'adult': False, 'gender': 2, 'id': 1205, 'kn...","[{'adult': False, 'gender': 2, 'id': 376, 'kno...",41.0,...,NaN,Original,"[{'id': 549, 'name': 'prostitute'}, {'id': 592...",https://image.tmdb.org/t/p/w185/hVHUfT801LQATG...,3.0,TF1,2013-02-21 00:00:00,Etats-Unis,"21,8","47,3"
1048,37652,118976,Didier,['Comedy'],1997-01-28,FR,104,"[{'adult': False, 'gender': 2, 'id': 28281, 'k...","[{'adult': False, 'gender': 2, 'id': 4275, 'kn...",46.0,...,NaN,Original,"[{'id': 6075, 'name': 'sports'}, {'id': 15162,...",https://image.tmdb.org/t/p/w185/aS8gMAIjuxCroS...,NaN,TF1,2013-05-18 00:00:00,France,"19,9","48,1"
1049,24260,80929,Inspector Blunder,['Comedy'],1980-12-03,FR,100,"[{'adult': False, 'gender': 2, 'id': 11214, 'k...","[{'adult': False, 'gender': 2, 'id': 1657, 'kn...",36.0,...,NaN,Original,[],https://image.tmdb.org/t/p/w185/m4xBYgr8hnrZaj...,NaN,TF1,2013-01-12 00:00:00,France,"19,8","44,1"
1050,6951,98536,Turner & Hooch,"['Action', 'Comedy', 'Thriller', 'Crime', 'Fam...",1989-07-28,Inconnu,100,"[{'adult': False, 'gender': 2, 'id': 31, 'know...","[{'adult': False, 'gender': 2, 'id': 598, 'kno...",33.0,...,NaN,Original,"[{'id': 15167, 'name': 'police detective'}, {'...",https://image.tmdb.org/t/p/w185/g3LC4mc17fQIFn...,1.0,M6,2013-02-28 00:00:00,Etats-Unis,"11,6","25,4"


In [8]:
path = os.path.join(chemin_donnees, "Lucas.csv")
df_final.to_csv(path, index=False)